In [18]:
from spaco_py.SpaCoObject import SPACO
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from momentchi2 import wf, hbe

In [2]:
sf = np.load(
    "/home/krastegar0/SpaCo_py/src/spaco_py/sf_mat.npy", allow_pickle=False
)
neighbor = np.load(
    "/home/krastegar0/SpaCo_py/src/spaco_py/A_mat.npy", allow_pickle=False
)

In [3]:
#Loading the coorddinates 
coords = pd.read_excel('/home/krastegar0/SpaCo_py/mouseBrainTestData/coords_brain.xlsx')

In [39]:
def plot_spatial_heatmap(coords, values, title="Spatial Heatmap", cmap="viridis", point_size=50):
    """
    Plots a discrete heatmap based on coordinates and corresponding values.
    Args:
        coords (numpy.ndarray): Array of shape (n_samples, 2) containing x and y coordinates.
        values (numpy.ndarray): Array of shape (n_samples,) containing values corresponding to the coordinates.
        title (str): Title of the plot.
        cmap (str): Colormap to use for the heatmap.
        point_size (int): Size of the points in the scatter plot.
    """
    # Ensure inputs are NumPy arrays
    coords = np.array(coords)
    values = np.array(values)
    # Create the scatter plot
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(coords[:, 0], coords[:, 1], c=values, cmap=cmap, s=point_size, edgecolor="k")
    plt.colorbar(scatter, label="Values")
    plt.title(title)
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.grid(True)
    plt.savefig(f'{title}.png', dpi=300, bbox_inches='tight')
    plt.show()
# Example usage
# Coordinates (x, y)
# coords = coords
# Values corresponding to the coordinates
# values = spac_patterns_X[:, 0]  # Use the first SpaCo pattern for demonstration
# Plot the heatmap
# plot_spatial_heatmap(coords, values, title="First SpaC",point_size=20)

In [5]:
testObj = SPACO(neighbormatrix=neighbor, sample_features=sf)

In [6]:
Pspac, Vkt =testObj.spaco_projection()

Inital CI: 1.766 - 1.768

INITIAL # of elements in CI: 0
 size of CI: 0.00176


In [7]:
non_random_eigvals = testObj._cache["non_random_eigvals"]
random_permuted_eigvals = testObj._cache["results_all"]


## Remarks on Fig 1
The distribution of eigenvalues shows the non-random eigenvalues in blue and the random permuted eigenvalues in orange.
The non-random eigenvalues represent the true structure of the data, while the random permuted eigenvalues serve as a null distribution for comparison.
We use the upper 95% confidence interval of the random permuted eigenvalues to determine the significance of the non-random eigenvalues.
The non-random eigenvalues that exceed this threshold are considered significant and indicate the presence of meaningful patterns in the data.
The plot provides a visual representation of the eigenvalue distributions, allowing us to assess the significance of the non-random eigenvalues in relation to the random permuted eigenvalues.
      
      

## Determing a metric to see if this pattern is actually spatially 
Here we see our first spatial pattern but how can we tell if the patterns that we see are actually spatial or they are just artifacts. We look at the length of the projections as the measure of "Smoothness" for a spatial gene 

In [10]:
def spaco_test(self, x: np.ndarray) -> float:
    """
    Compute the spatially variable test statistic for any given input vector x.

    Returns
    -------
    sigma: np.ndarray
        The eigenvalues of the transformed matrix.
    """

    # Scaling input vector (should this just be centered and not scaled? )
    gene: np.ndarray = (x - x.mean()) / x.std()
    assert np.isclose(gene.mean(), 0, atol=1e-8) and np.isclose(
        gene.std(), 1, atol=1e-8
    ), "Gene is not centered and scaled"

    # Compute the eigenvalues of the transformed matrix and the graph Laplacian (L)
    sorted_sigma_eigh = self.sigma_eigh[::-1]
    # printing all the variables berfore the test statistic

    # Normalize the scaled data
    gene = gene / np.repeat(np.sqrt(gene.T @ self.graphLaplacian @ gene), len(gene))
    print(f"sigma: {self.sigma.shape}\ngene: {gene.shape}")
    # Compute the test statistic
    test_statistic: float = float(gene.T @ self.sigma @ gene)
    # print(f'test statistic: {test_statistic}\n\n\n sorted_sigma_eigenvals: {sorted_sigma_eigh[:nSpacs]}')
    # pval test statistic
    pVal: float = self.__psum_chisq(
        q=test_statistic, eig_vals=sorted_sigma_eigh[: self.Vk.shape[1]]
    )
    print(
        f"pval: {pVal}\ntest statistic: {test_statistic}\nlambda cut: {self.lambda_cut}"
    )
    return pVal, test_statistic

In [11]:
# need to generate the sigma matrix and sigma eigenvlaues for test statistic using 
testObj.spaco_test(Pspac[:, 0])

sigma: (2696, 2696)
gene: (2696,)
pval: -0.4456515974592101
test statistic: 1.0000000000000002
lambda cut: None


(-0.4456515974592101, 1.0000000000000002)

In [12]:
coeff=testObj._cache["sigma_eigh"]

In [13]:
Q_sig=testObj._cache["sigma"]

### Going to use python momentchi2 package to run the weighted sum of chi2 distribution 
Here we attempt to use a weighted sum of chi2 package to determine if the length of the projected spaco pattern relates to a spatially relevant gene 

In [35]:
gene = Vkt[:, 1]
#gene = gene / np.repeat(np.sqrt(gene.T @ testObj.graphLaplacian @ gene), len(gene))
test_statistic: float = gene.T @ Q_sig @ gene
sorted_sigma_eigh = coeff[::-1]
non_zero_coef = sorted_sigma_eigh[sorted_sigma_eigh > 0]
# pVal test statistic
pval=wf(non_zero_coef, test_statistic)
pval

np.float64(0.9999999999999999)

In [36]:
test_statistic

np.float64(1.0000000000000013)